In [1]:
%run ../yp_utils.py

# Initial setup

In [2]:
paper_pmid = 27909446
paper_name = 'north_vulpe_2016' 

In [3]:
datasets = pd.read_csv('extras/YeastPhenome_' + str(paper_pmid) + '_datasets_list.txt', sep='\t', header=None, names=['dataset_id', 'name'])

In [4]:
datasets.set_index('dataset_id', inplace=True)

# Load & process the data

In [24]:
original_data = pd.read_excel('raw_data/Table1.xlsx', sheet_name='Strains with altered growth in ', skiprows=10)

In [25]:
print('Original data dimensions: %d x %d' % (original_data.shape))

Original data dimensions: 742 x 9


In [26]:
original_data.head()

,ORF,Gene name,150μM,300μM,600μM,150μM.1,300μM.1,600μM.1,Description
0,YCR060W,TAH1,1.7,1.9,2.3,1.7,1.8,1.6,"HSP90 cofactor; interacts with Hsp82p, Pih1p, ..."
1,YAR044W,NaN,2.0,1.8,3.2,2.0,2.1,2.3,NaN
2,YAR031W,PRM9,2.2,2.1,2.4,1.7,1.7,1.7,Pheromone-regulated protein with 3 predicted t...
3,YBR114W,RAD16,2.3,2.5,2.8,2.4,2.3,2.4,Protein that recognizes and binds damaged DNA ...
4,YMR261C,TPS3,2.7,2.7,2.9,2.3,2.3,2.3,Regulatory subunit of trehalose-6-phosphate sy...


In [27]:
original_data['orf'] = original_data['ORF'].astype(str)

In [28]:
# Eliminate all white spaces & capitalize
original_data['orf'] = clean_orf(original_data['orf'])

In [29]:
# Translate to ORFs 
original_data['orf'] = translate_sc(original_data['orf'], to='orf')

In [30]:
# Make sure everything translated ok
t = looks_like_orf(original_data['orf'])
print(original_data.loc[~t,])

Empty DataFrame
Columns: [ORF, Gene name, 150μM, 300μM, 600μM, 150μM.1, 300μM.1, 600μM.1, Description, orf]
Index: []


In [31]:
original_data.set_index('orf', inplace=True)

In [32]:
original_data = original_data[['150μM','300μM','600μM','150μM.1','300μM.1','600μM.1']].copy()

In [33]:
original_data = original_data.groupby(original_data.index).mean()

In [34]:
original_data.shape

(742, 6)

# Prepare the final dataset

In [35]:
data = original_data.copy()

In [36]:
dataset_ids = [5377, 5376, 5375, 5380, 5379, 5378]
datasets = datasets.reindex(index=dataset_ids)

In [37]:
lst = [datasets.index.values, ['value']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data.columns = idx

In [39]:
data[data.isnull()] = 0

In [40]:
data.head()

dataset_id,5377,5376,5375,5380,5379,5378
data_type,value,value,value,value,value,value
orf,,,,,,
YAL001C,1.7,0.0,0.0,0.0,0.0,0.0
YAL015C,0.0,0.0,0.0,0.0,0.0,-1.7
YAL027W,0.0,0.0,0.0,-1.1,-1.1,-1.0
YAL031C,0.0,0.0,2.1,0.0,0.0,0.0
YAL039C,0.0,0.0,2.3,0.0,0.0,0.0


## Subset to the genes currently in SGD

In [41]:
genes = pd.read_csv(path_to_genes, sep='\t', index_col='id')
genes = genes.reset_index().set_index('systematic_name')
gene_ids = genes.reindex(index=data.index.values)['id'].values
num_missing = np.sum(np.isnan(gene_ids))
print('ORFs missing from SGD: %d' % num_missing)

ORFs missing from SGD: 2


In [42]:
data['gene_id'] = gene_ids
data = data.loc[data['gene_id'].notnull()]
data['gene_id'] = data['gene_id'].astype(int)
data = data.reset_index().set_index(['gene_id','orf'])

In [43]:
data.head()

,dataset_id,5377,5376,5375,5380,5379,5378
,data_type,value,value,value,value,value,value
gene_id,orf,,,,,,
1,YAL001C,1.7,0.0,0.0,0.0,0.0,0.0
13,YAL015C,0.0,0.0,0.0,0.0,0.0,-1.7
25,YAL027W,0.0,0.0,0.0,-1.1,-1.1,-1.0
29,YAL031C,0.0,0.0,2.1,0.0,0.0,0.0
37,YAL039C,0.0,0.0,2.3,0.0,0.0,0.0


# Normalize

In [44]:
data_norm = normalize_phenotypic_scores(data, has_tested=False)

In [45]:
# Assign proper column names
lst = [datasets.index.values, ['valuez']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data_norm.columns = idx

In [46]:
data_norm[data.isnull()] = np.nan
data_all = data.join(data_norm)

In [47]:
data_all.head()


,dataset_id,5377,5376,5375,5380,5379,5378,5377,5376,5375,5380,5379,5378
,data_type,value,value,value,value,value,value,valuez,valuez,valuez,valuez,valuez,valuez
gene_id,orf,,,,,,,,,,,,
1,YAL001C,1.7,0.0,0.0,0.0,0.0,0.0,3.542288,-0.013987,0.064552,0.005930,-0.087678,0.045892
13,YAL015C,0.0,0.0,0.0,0.0,0.0,-1.7,0.055931,-0.013987,0.064552,0.005930,-0.087678,-3.172260
25,YAL027W,0.0,0.0,0.0,-1.1,-1.1,-1.0,0.055931,-0.013987,0.064552,-2.146564,-2.416485,-1.847139
29,YAL031C,0.0,0.0,2.1,0.0,0.0,0.0,0.055931,-0.013987,4.131350,0.005930,-0.087678,0.045892
37,YAL039C,0.0,0.0,2.3,0.0,0.0,0.0,0.055931,-0.013987,4.518664,0.005930,-0.087678,0.045892


# Print out

In [48]:
for f in ['value','valuez']:
    df = data_all.xs(f, level='data_type', axis=1).copy()
    df.columns = datasets['name'].values
    df = df.droplevel('gene_id', axis=0)
    df.to_csv(paper_name + '_' + f + '.txt', sep='\t')

# Save to DB

In [49]:
from IO.save_data_to_db3 import *

In [50]:
save_data_to_db(data_all, paper_pmid)

  0%|          | 0/6 [00:00<?, ?it/s]

Deleting all datasets for PMID 27909446...
Inserting the new data...


100%|██████████| 6/6 [00:07<00:00,  1.23s/it]

Updating the data_modified_on field...
